In [1]:
# 완전한 비동기 Tesla 뉴스 스크래핑 Agent

import nest_asyncio
import asyncio
from playwright.async_api import async_playwright
from langchain_core.tools import tool
from typing import Annotated, List, Dict
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import ToolMessage, HumanMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
import os

# 이벤트 루프 패치
nest_asyncio.apply()

# API 키 설정
os.environ["GOOGLE_API_KEY"] = 'your api key'


In [2]:
# 비동기 스크래핑 도구 정의
@tool
async def scrape_articles_with_content(query: str, max_articles: int = 3) -> str:
    """
    Econotimes에서 관련 기사 제목, URL, 본문을 스크랩하는 비동기 함수
    사용자 특정 종목에 대한 동향 분석 등을 요청할 때 이 도구를 이용해 뉴스 기사를 검색합니다.
    
    
    Args:
        query: 검색할 키워드 (예: tesla, apple, bitcoin 등)
        max_articles: 추출할 최대 기사 수 (기본값: 3)
    
    Returns:
        기사 정보가 포함된 JSON 형태의 문자열
    """
    print(f"🚀 Econotimes에서 '{query}' 검색 중...")
    
    output_list = []
    
    try:
        async with async_playwright() as p:
            # 브라우저 실행
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            
            # 검색 페이지로 이동
            search_url = f"https://econotimes.com/search?v={query}&search="
            await page.goto(search_url)
            await asyncio.sleep(2)
            
            # XPath를 사용해서 모든 기사 제목 요소 찾기
            general_xpath = '//*[@id="archivePage"]/div/div[2]/div/p[1]/a'
            elements = await page.locator(f"xpath={general_xpath}").all()
            
            
            if not elements:
                await browser.close()
                return f"'{query}'에 대한 기사를 찾을 수 없습니다."
            
            # 지정된 개수만큼 기사 처리
            for i, element in enumerate(elements[:max_articles], 1):
                try:
                    # 기사 제목과 링크 추출
                    title = await element.text_content()
                    href = await element.get_attribute('href')
                    
                    if title and href:
                        title = title.strip()
                        full_url = f"https://econotimes.com{href}" if href.startswith('/') else href
                        
                        print(f"{i}. {title}")
                        
                        # 새 탭에서 기사 본문 추출
                        article_page = await browser.new_page()
                        try:
                            await article_page.goto(full_url)
                            await asyncio.sleep(2)
                            
                            # 본문 추출
                            article_xpath = '//*[@id="view"]/div[2]/div[3]/article'
                            article_content = await article_page.locator(f"xpath={article_xpath}").text_content()
                            
                            if article_content:
                                article_content = article_content.strip()
                                # 본문이 너무 길면 앞부분만
                                content_preview = article_content[:800] + "..." if len(article_content) > 800 else article_content
                            else:
                                content_preview = "본문을 추출할 수 없습니다."
                            
                            output_list.append({
                                'number': i,
                                'title': title,
                                'url': full_url,
                                'content': content_preview
                            })
                            
                            
                            
                        except Exception as e:
                            print(f"   ❌ 본문 추출 실패: {e}")
                            output_list.append({
                                'number': i,
                                'title': title,
                                'url': full_url,
                                'content': '본문 추출 실패'
                            })
                        finally:
                            await article_page.close()
                
                except Exception as e:
                    print(f"{i}. ❌ 기사 처리 실패: {e}")
                    continue
            
            await browser.close()
            
            if output_list:

                # JSON 형태로 반환
                import json
                return json.dumps(output_list, ensure_ascii=False, indent=2)
            else:
                return f"'{query}' 기사 추출에 실패했습니다."
                
    except Exception as e:
        return f"스크래핑 중 오류 발생: {str(e)}"


In [3]:
# 상태 정의 및 LLM 설정
class State(TypedDict):
    messages: Annotated[list, add_messages]

# LLM 설정
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# 도구 바인딩
tools = [scrape_articles_with_content]
llm_with_tools = llm.bind_tools(tools)


In [7]:
# 비동기 노드 정의
def chatbot_node(state: State):
    """LLM이 응답하는 노드"""
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

async def async_tool_node(state: State):
    """비동기 도구를 실행하는 노드"""
    message = state["messages"][-1]
    outputs = []
    
    for tool_call in message.tool_calls:
        print(f"🔧 도구 실행: {tool_call['name']}")
        
        # 비동기 도구 실행
        if tool_call["name"] == "scrape_articles_with_content":
            result = await scrape_articles_with_content.ainvoke(tool_call["args"])
        else:
            result = "알 수 없는 도구입니다."
        
        outputs.append(
            ToolMessage(
                content=str(result),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    
    return {"messages": outputs}

def route_tools(state: State):
    """도구를 사용할지 결정하는 라우팅 함수"""
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END


In [8]:
# 그래프 구성 및 컴파일
graph_builder = StateGraph(State)

# 노드 추가
graph_builder.add_node("chatbot", chatbot_node)
graph_builder.add_node("tools", async_tool_node)

# 엣지 설정
graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    {"tools": "tools", END: END},
)
graph_builder.add_edge("tools", "chatbot")

# 그래프 컴파일
graph = graph_builder.compile()



In [9]:
# 비동기 스트리밍 실행 함수
async def stream_graph_updates_verbose(user_input: str):
    """모든 중간 단계를 자세히 출력하는 비동기 함수"""
    inputs = {"messages": [HumanMessage(content=user_input)]}

    print("--- Agent Start ---")
    async for event in graph.astream(inputs, stream_mode="values"):
        message = event["messages"][-1]
        
        # AIMessage이면서 tool_calls가 있는 경우 (계획 단계)
        if isinstance(message, AIMessage) and message.tool_calls:
            print(f"🤖 [AIMessage]: Tool Call Planned -> {message.tool_calls[0]['name']}({message.tool_calls[0]['args']})")
        
        # ToolMessage인 경우 (실행 결과)
        elif isinstance(message, ToolMessage):
            content = str(message.content)
            if content.startswith('[') and len(content) > 100:
                print(f"🛠️ [ToolMessage]: 스크래핑 완료! 기사 데이터 추출됨")
            else:
                print(f"🛠️ [ToolMessage]: {content[:100]}...")

        # AIMessage이면서 최종 답변인 경우
        elif isinstance(message, AIMessage) and message.content:
            print(f"💬 [AIMessage]: Final Answer -> {message.content}")
    print("--- Agent End ---")

# 메인 실행 함수 (비동기)
async def main():
    print("뉴스 스크래핑 Agent 시작")
    
    while True:
        try:
            user_input = input("\nUser: ")
            if user_input.lower() in ["quit", "exit", "q", "종료"]:
                print("Goodbye! 👋")
                break
            
            # 비동기 함수 실행
            await stream_graph_updates_verbose(user_input)
            
        except (KeyboardInterrupt, EOFError):
            print("\nGoodbye! 👋")
            break
        except Exception as e:
            print(f"\n❌ 오류가 발생했습니다: {e}\n")

# 비동기 실행
await main()


뉴스 스크래핑 Agent 시작



User:  samsung 뉴스 검색 후 동향 분석해줘


--- Agent Start ---
🤖 [AIMessage]: Tool Call Planned -> scrape_articles_with_content({'query': 'samsung'})
🔧 도구 실행: scrape_articles_with_content
🚀 Econotimes에서 'samsung' 검색 중...
1. U.S. Plans Annual Export Approvals for Samsung and SK Hynix Chip Supplies in China
2. Samsung Electronics Q2 Profit Plunges 55% Amid Memory Chip Delays, U.S. Export Curbs
3. Samsung Shares Dip After Rally on $16.5B Tesla Chip Deal
🛠️ [ToolMessage]: 스크래핑 완료! 기사 데이터 추출됨
💬 [AIMessage]: Final Answer -> 삼성전자의 최근 동향은 다음과 같습니다.

*   **미국의 수출 규제 강화**: 미국은 삼성전자와 SK하이닉스가 중국 내 공장에 칩 제조 장비를 수출할 때 매년 승인을 받도록 하는 새로운 규정을 검토 중입니다. 이는 기존의 무기한 승인에서 연간 검토 방식으로 전환되어, 미국이 중국으로의 첨단 칩 제조 도구 공급에 대한 통제력을 강화하려는 움직임으로 보입니다.
*   **2분기 실적 부진**: 삼성전자는 고대역폭 메모리(HBM) 칩 출하 지연과 미국의 대중국 반도체 수출 규제로 인해 2분기 영업이익이 55% 감소했습니다. 이는 차세대 HBM 칩 시장에서의 경쟁력과 관련된 투자자들의 우려를 키우고 있습니다.
*   **테슬라와의 대규모 칩 공급 계약**: 테슬라의 차세대 AI6 칩 공급을 위한 165억 달러 규모의 계약을 체결했습니다. 이 계약은 삼성전자가 전기차 및 AI 칩 시장에서 중요한 역할을 할 수 있음을 보여주며, 파운드리 사업에 긍정적인 모멘텀을 제공할 것으로 예상됩니다.

요약하자면, 삼성전자는 미국의 대


User:  q


Goodbye! 👋


In [ ]:
samsung 뉴스 검색 후 동향 분석해줘